In [9]:
import pandas as pd
import re

df = pd.read_csv('/home/yl3427/cylab/llm_reasoning/reasoning/data/step2_ALL.csv', encoding="latin-1")
df

,ï»¿idx,question,choice,ground_truth,qn_num
0,1,A 21-year-old man comes to student health serv...,"A: Agoraphobia, B: Generalized anxiety disorde...",A,1
1,2,A 35-year-old man is brought to the emergency ...,"A: denosine stress test, B: Cardiac catheteriz...",E,2
2,3,A 65-year-old man comes to the office because ...,"A: Amiodarone, B: Lisinopril, C: Propranolol, ...",A,3
3,4,An 18-month-old boy is brought to the emergenc...,"A: Audiography, B: DEXA scan, C: Echocardiogr...",A,4
4,5,A 70-year-old woman is admitted to the hospita...,"A: Boyfriend, B: Neighbor, C: Patient, D: Phys...",E,5
...,...,...,...,...,...
115,116,A hospitalized 37-year-old woman has a 2-hour ...,"A: Air embolism; B: Empyema, C: Hemothorax, D:...",H,116
116,117,A 77-year-old man is brought to the emergency ...,"A: Acute pancreatitis, B: Mesenteric ischemia,...",D,117
117,118,A 1-week-old male newborn is evaluated in the ...,"A: Congenital heart malformation, B: Cryptorch...",D,118
118,119,A randomized controlled trial is conducted to ...,"A: Extrapolation of findings beyond data, B: I...",A,119


In [3]:
df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv',
                        usecols=['File ID', 'Subjective', 'Objective', 'Assessment', 'Summary', 'cleaned_expanded_Summary', 'terms'])

df = df.fillna('').apply(lambda x: x.str.lower())
df['combined_summary'] = df['Summary'] + df['cleaned_expanded_Summary'] + df['terms']

mi = ["myocardial infarction", "elevation mi", "non-stemi", " NSTEMI", " stemi"]
chf = ["congestive heart failure", " chf", "HFrEF", "HFpEF"]
pulmonary_embolism = ["pulmonary embolism"]
pulmonary_hypertension = ["pulmonary hypertension", "pulmonary htn"]
sepsis = ["sepsis", "septic shock"]
urosepsis = ["urosepsis"]
meningitis = ["meningitis"]
aki = ["acute kidney injury", " aki", "acute renal failure", " arf"] # -> Acute tubular necrosis (ATN)인가 아닌가
atn = ["acute tubular necrosis", " atn"]
pancreatitis = ["pancreatitis"]
gi_bleed = ["gastrointestinal bleed", "gi bleed"]
hepatitis = ["hepatitis", " hep"]
cholangitis = ["cholangitis"]
asp_pneumonia = ["aspiration pneumonia"]

prob_dict = {'sepsis': sepsis, 
                'acute kidney injury': aki, 
                'pancreatitis': pancreatitis, 
                'gastrointestinal bleed': gi_bleed,
                "congestive heart failure": chf}

ids = set()
for name, lst in prob_dict.items():
    problem_terms = lst
    problem_terms = [term.lower() for term in problem_terms]

    # Use the first term as the primary term to check in the combined summary.
    primary_term = problem_terms[0]

    # Build a regex pattern that matches any of the problem terms.
    # pattern = '|'.join(problem_terms)
    pattern = '|'.join(re.escape(term) for term in problem_terms)

    mask = (
        df['combined_summary'].str.contains(pattern, na=False) &
        ~df['Subjective'].str.contains(pattern, na=False) &
        ~df['Objective'].str.contains(pattern, na=False)
    )

    filtered_df = df[mask]

    ids.update(filtered_df['File ID'])

df = df[df['File ID'].isin(ids)]
df.to_csv('/home/yl3427/cylab/SOAP_MA/Input/SOAP_5_problems.csv', index=False)
print(df.shape)

(351, 8)
